# Practical machine learning and deep learning. Lab 5
## Competition
No competition for today



# Fine-tuning a model on a translation task
Today we will be finetunning T5(Text-To-Text Transfer Transformer) [model](https://github.com/google-research/t5x) on translation task. For this purpose we will be using [HuggingFace transformers](https://huggingface.co/docs/transformers/index) and [WMT16](https://huggingface.co/datasets/wmt16) dataset. 

In [1]:
# installing huggingface libraries for dataset, models and metrics
%pip install datasets transformers[sentencepiece] sacrebleu

%pip install numpy==1.24.3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Necessary inputs
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

## Selecting the model
For the example purpose we select as model checkpoint the smallest transformer in T5 family - `t5_small`. Other pre-trained models can be found [here](https://huggingface.co/docs/transformers/model_doc/t5#:~:text=T5%20comes%20in%20different%20sizes%3A).

In [3]:
# selecting model checkpoint
model_checkpoint = "t5-small"

## Loading the dataset

In [4]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the WMT16 dataset
raw_datasets = load_dataset("wmt16", "de-en")

# Load the BLUE metric
metric = load_metric("sacrebleu")

## Dataset
Downloaded from HuggingFace dataset is a `DatasetDict`. It contains keys `["train", "validation", "test"]` - which represents a dataset splits

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

In [6]:
# samples from train dataset
raw_datasets["train"][:5]

{'translation': [{'de': 'Wiederaufnahme der Sitzungsperiode',
   'en': 'Resumption of the session'},
  {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
   'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'},
  {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
   'en': "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."},
  {'de': 'Im Parlament besteht der Wunsch na

## Metric
[Sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu) computes:
- `score`: BLEU score
- `counts`: list of counts of correct n-grams
- `totals`: list of counts of total n-grams
- `precisions`: list of precisions
- `bp`: Brevity penalty
- `sys_len`: cumulative sysem length
- `ref_len`: cumulative reference length

The main metric is [BLEU score](https://en.wikipedia.org/wiki/BLEU). BLEU (BiLingual Evaluation Understudy) is a metric for automatically evaluating machine-translated text. The BLEU score measures the similarity of the machine-translated text to a set of high quality reference translations.

The BLEU metric is calculates using [n-grams](https://en.wikipedia.org/wiki/N-gram).

In [7]:
# how to use sacrebleu and its purpose
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [8]:
fake_preds = ["hello there", "general kenobi", "Can I get an A"]
fake_labels = [["hello there"], ["general kenobi"], ['Can I get a C']]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 45.59274666224604,
 'counts': [7, 4, 1, 0],
 'totals': [9, 6, 3, 2],
 'precisions': [77.77777777777777,
  66.66666666666667,
  33.333333333333336,
  25.0],
 'bp': 1.0,
 'sys_len': 9,
 'ref_len': 9}

## Preprocessing the data
As usual we will need to preprocess data and tokenize it before passing to model

In [9]:
from transformers import AutoTokenizer

# we will use autotokenizer for this purpose
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
max_input_length = 1024
max_target_length = 1024
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
# for the example purpose we will crop the dataset and select first 5000 for train
# and 500 for validation and test
cropped_datasets = raw_datasets
cropped_datasets['train'] = raw_datasets['train'].select(range(5000))
cropped_datasets['validation'] = raw_datasets['validation'].select(range(500))
cropped_datasets['test'] = raw_datasets['test'].select(range(500))
tokenized_datasets = cropped_datasets.map(preprocess_function, batched=True)
tokenized_datasets['train'][0]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

NameError: name 'prefix' is not defined

## Fine-tuning the model

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [13]:
def process_prompt(model, prompt, tokenizer=tokenizer):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True, temperature=0))

def get_process_prompt(model, tokenizer=tokenizer):
    def callback(prompt):
        process_prompt(model, prompt, tokenizer)
    
    return callback

In [14]:
ask = get_process_prompt(model)

In [20]:
detox = "I like that shit."

ask("repharse:" + detox)